References:

- Standardized computer-based organized reporting of EEG:SCORE – Second version
Beniczkya et al., 



In [ ]:
import sys
print(sys.executable)
#! /Users/shek/anaconda3/envs/medcatenv/bin/python -m pip install adjustText

In [18]:
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import Mapping
from adjustText import adjust_text


# Terminology

In [19]:
eeg_terminiology = {
    'Root_EEG_concept': {
        'Location of graphelements':{
            'laterality':{
                'left':[],
                'right':[],
                'midline':[],
                'bilateral':[],
                'diffuse':[],
            },
            'region':{
                'frontal':{
                    'perisylvian-superior surface':[],
                    'lateral':[],
                    'mesial':[],
                    'polar':[],
                    'orbitofrontal':[],
                },
                'central':{
                    'lateral convexity':[],
                    'mesial':[],
                    'central sulcus -anterior surface':[],
                    'central sulcus -posterior surface':[],
                    'opercular':[],
                },
                'temporal':{
                    'polar':[],
                    'basal':[],
                    'lateral-anterior':[],
                    'lateral-posterior':[],
                    'perisylvian-inferior surface':[],
                },
                'parietal':{
                    'lateral-convexity':[],
                    'mesial':[],
                    'opercular':[],
                },
                'occipital':{
                    'lateral':[],
                    'mesial':[],
                    'basal':[],
                },
                'insula':[],
            },
        },
        'Interical findings':{
            'Epileptiform interictal activity':{
                'Spike':[],
                'Spike and slow wave':[],
                'Runs of rapid spikes':[],
                'Polyspikes':[],
                'Polyspike and slow wave':[],
                'Sharp wave':[],
                'Sharp and slow wave':[],
                'High frequency oscillations (HFO)':[],
                'Hypsarrhythmia - classic':[],
                'Hypsarrythmia - modified':[],
            }, 
            'Abnormal interictal rhythmic activity':{
                 'Delta activity':[],
                 'Theta activity':[],
                 'Alpha activity':[],
                 'Beta activity':[],
                 'Gamma activity':[],
                 'Polymorphic delta':[],
                 'Frontal intermittent rhythmic delta activity (FIRDA)':[],
                 'Occipital intermittent rhythmic delta activity (OIRDA)':[],
                 'Temporal intermittent rhythmic deltaactivity (TIRDA)':[],
             },
            'Special patterns':{
                'Periodic discharges not further specified (PDs)':[],
                'Generalized periodic discharges (GPDs)':[],
                'Lateralized periodic discharges (LPDs)':[],
                'Bilateral independent periodic discharges (BIPDs)':[],
                'Multifocal periodic discharges (MfPDs)':[],
                'Extreme delta brush':[],
                'Burst suppression':[],
                'Burst attenuation':[],
            },
        },
        'Icatal EEG findings':{
            'No observable change':[],
            'Obscured by artifacts':[],
            'Polyspikes':[],
            'Fast spike activity or repetitive spikes':[],
            'Low voltage fast activity':[],
            'Polysharp-waves':[],
            'Spike-and-slow-waves':[],
            'Polyspike-and-slow-waves':[],
            'Sharp-and-slow-waves':[],
            'Rhythmic activity':[],
            'Slow wave of large amplitude':[],
            'Irregular delta or theta activity':[],
            'Burst-suppression pattern':[],
            'Electrodecremental change':[],
            'DC-shift':[],
            'High frequency oscillation (HFO)':[],
            'Disappearance of ongoing activity':[],
        },
        'Patterns not considered abnormal':{
            'Physiologic patterns':{
                'Rhythmic activity':[],
                'Slow alpha variant rhythms':[],
                'Fast alpha variant rhythms':[],
                'Frontocentral theta activity':[],
                'Lambda waves':[],
                'Posterior slow waves in youth':[],
                'Diffuse low frequency activity induced by hyperventilation':[],
                'Photic drive response':[],
                'Photomyogenic response(orbitofrontal photomyoclonus)':[],
                'Arousal pattern':[],
                'Frontal arousal rhythm':[], 
            },
            'Patterns of uncertain significance':{
                'Sharp transient Wicket spikes':[],
                'Small sharp spikes (Benign Epileptiform Transients of Sleep)':[],
                'Rhythmic temporal theta burst of drowsiness':[],
                'Ciganek rhythm (midline central theta)':[],
                '6 Hz spike and slow wave':[],
                '14 and 6 Hz positive bursts':[],
                'Rudimentary spike wave complex':[],
                'Slow fused transient':[],
                'Needle-like occipital spikes of the blind':[],
                'Subclinical Rhythmic EEG Dischargesin Adults (SREDA)':[],
                'Temporal slowing in elderly subjects Breach rhythm':[],
            },
        },
    },
}
    
    

# Visualisation of temininology

In [20]:
G = nx.DiGraph()

In [21]:
q = list(eeg_terminiology.items())
while q:
    v, d = q.pop()
    for nv, nd in d.items():
        G.add_edge(v, nv)
        if isinstance(nd, Mapping):
            q.append((nv, nd))
            
np.random.seed(42)

In [22]:
list(G.edges)

[('Root_EEG_concept', 'Location of graphelements'),
 ('Root_EEG_concept', 'Interical findings'),
 ('Root_EEG_concept', 'Icatal EEG findings'),
 ('Root_EEG_concept', 'Patterns not considered abnormal'),
 ('Location of graphelements', 'laterality'),
 ('Location of graphelements', 'region'),
 ('Interical findings', 'Epileptiform interictal activity'),
 ('Interical findings', 'Abnormal interictal rhythmic activity'),
 ('Interical findings', 'Special patterns'),
 ('Icatal EEG findings', 'No observable change'),
 ('Icatal EEG findings', 'Obscured by artifacts'),
 ('Icatal EEG findings', 'Polyspikes'),
 ('Icatal EEG findings', 'Fast spike activity or repetitive spikes'),
 ('Icatal EEG findings', 'Low voltage fast activity'),
 ('Icatal EEG findings', 'Polysharp-waves'),
 ('Icatal EEG findings', 'Spike-and-slow-waves'),
 ('Icatal EEG findings', 'Polyspike-and-slow-waves'),
 ('Icatal EEG findings', 'Sharp-and-slow-waves'),
 ('Icatal EEG findings', 'Rhythmic activity'),
 ('Icatal EEG findings', '

In [23]:
def hierarchy_pos(G, root, levels=None, width=1., height=1.):
    '''If there is a cycle that is reachable from root, then this will see infinite recursion.
       G: the graph
       root: the root node
       levels: a dictionary
               key: level number (starting from 0)
               value: number of nodes in this level
       width: horizontal space allocated for drawing
       height: vertical space allocated for drawing'''
    TOTAL = "total"
    CURRENT = "current"
    def make_levels(levels, node=root, currentLevel=0, parent=None):
        """Compute the number of nodes for each level
        """
        if not currentLevel in levels:
            levels[currentLevel] = {TOTAL : 0, CURRENT : 0}
        levels[currentLevel][TOTAL] += 1
        neighbors = G.neighbors(node)
        for neighbor in neighbors:
            if not neighbor == parent:
                levels =  make_levels(levels, neighbor, currentLevel + 1, node)
        return levels

    def make_pos(pos, node=root, currentLevel=0, parent=None, vert_loc=0):
        dx = 1/levels[currentLevel][TOTAL]
        left = dx/2
        pos[node] = ((left + dx*levels[currentLevel][CURRENT])*width, vert_loc)
        levels[currentLevel][CURRENT] += 1
        neighbors = G.neighbors(node)
        for neighbor in neighbors:
            if not neighbor == parent:
                pos = make_pos(pos, neighbor, currentLevel + 1, node, vert_loc-vert_gap)
        return pos
    if levels is None:
        levels = make_levels({})
    else:
        levels = {l:{TOTAL: levels[l], CURRENT:0} for l in levels}
    vert_gap = height / (max([l for l in levels])+1)
    return make_pos({})

In [24]:
pos = hierarchy_pos(G, 'Root_EEG_concept')

In [ ]:
plt.figure(3, figsize=(120,15))
plt.title("EEG Findings Terminology", fontsize=60)
nx.draw(G, pos=pos, with_labels=True, node_size=30,font_size=10,font_weight="bold",
        node_color="skyblue", node_shape="s", alpha=0.6, linewidths=40, 
        width = 0.5)
plt.tight_layout()
#plt.savefig("EEG_Findings_Terminology.png", dpi=300)


# Sunburst visualisation

follow tutorial:
https://notebook.community/empet/Plotly-plots/Radial-tree-US-Open-16.ipynb
https://community.plotly.com/t/radial-tree-with-plotly-and-networkx/12537 (walker layout)
https://chart-studio.plotly.com/~empet/14307/a-radial-tree/#/ (radial layout)

This one is good:
https://nbviewer.jupyter.org/github/empet/Plotly-Sunbursts/blob/master/Processing-tree-like-data-to-be-represented-as-a-SUNBURST.ipynb

In [1]:
import pygraphviz as pgv
import plotly.graph_objs as go
from chart_studio.plotly import iplot

# New terminology

In [26]:
eeg_terminiology ={
    'name':'Root_EEG', 
    'children':[
        {'name':'Location of graphelements', 
        'children':[
            {'name':'laterality', 
            'children':[
                {'name':'left'},
                {'name':'right'},
                {'name':'midline'},
                {'name':'bilateral'},
                {'name':'diffuse'},
                ]},
            {'name':'region',
             'children':[
                 {'name':'frontal',
                  'children':[
                    {'name':'perisylvian-superior surface'},
                    {'name':'lateral'},
                    {'name':'mesial'},
                    {'name':'polar'},
                    {'name':'orbitofrontal'},
                  ]},
                {'name':'central',
                 'children':[
                    {'name':'lateral convexity'},
                    {'name':'mesial'},
                    {'name':'central sulcus -anterior surface'},
                    {'name':'central sulcus -posterior surface'},
                    {'name':'opercular'},
                 ]},
                {'name':'temporal', 'children':[
                    {'name':'polar'},
                    {'name':'basal'},
                    {'name':'lateral-anterior'},
                    {'name':'lateral-posterior'},
                    {'name':'perisylvian-inferior surface'},
                ]},
                {'name':'parietal', 'children':[
                    {'name':'lateral-convexity'},
                    {'name':'mesial'},
                    {'name':'opercular'},
                ]},
                {'name':'occipital', 'children':[
                    {'name':'lateral'},
                    {'name':'mesial'},
                    {'name':'basal'},
                ]},
                {'name':'insula'},
            ]},
        ]},
        {'name':'Interical findings', 'children':[
            {'name':'Epileptiform interictal activity', 'children':[
                {'name':'Spike'},
                {'name':'Spike and slow wave'},
                {'name':'Runs of rapid spikes'},
                {'name':'Polyspikes'},
                {'name':'Polyspike and slow wave'},
                {'name':'Sharp wave'},
                {'name':'Sharp and slow wave'},
                {'name':'High frequency oscillations (HFO)'},
                {'name':'Hypsarrhythmia - classic'},
                {'name':'Hypsarrythmia - modified'},
            ]}, 
            {'name':'Abnormal interictal rhythmic activity', 'children':[
                 {'name':'Delta activity'},
                 {'name':'Theta activity'},
                 {'name':'Alpha activity'},
                 {'name':'Beta activity'},
                 {'name':'Gamma activity'},
                 {'name':'Polymorphic delta'},
                 {'name':'Frontal intermittent rhythmic delta activity (FIRDA)'},
                 {'name':'Occipital intermittent rhythmic delta activity (OIRDA)'},
                 {'name':'Temporal intermittent rhythmic deltaactivity (TIRDA)'},
             ]},
            {'name':'Special patterns', 'children':[
                {'name':'Periodic discharges not further specified (PDs)'},
                {'name':'Generalized periodic discharges (GPDs)'},
                {'name':'Lateralized periodic discharges (LPDs)'},
                {'name':'Bilateral independent periodic discharges (BIPDs)'},
                {'name':'Multifocal periodic discharges (MfPDs)'},
                {'name':'Extreme delta brush'},
                {'name':'Burst suppression'},
                {'name':'Burst attenuation'},
            ]},
        ]},
        {'name':'Ictal EEG findings', 'children':[
            {'name':'No observable change'},
            {'name':'Obscured by artifacts'},
            {'name':'Polyspikes'},
            {'name':'Fast spike activity or repetitive spikes'},
            {'name':'Low voltage fast activity'},
            {'name':'Polysharp-waves'},
            {'name':'Spike-and-slow-waves'},
            {'name':'Polyspike-and-slow-waves'},
            {'name':'Sharp-and-slow-waves'},
            {'name':'Rhythmic activity'},
            {'name':'Slow wave of large amplitude'},
            {'name':'Irregular delta or theta activity'},
            {'name':'Burst-suppression pattern'},
            {'name':'Electrodecremental change'},
            {'name':'DC-shift'},
            {'name':'High frequency oscillation (HFO)'},
            {'name':'Disappearance of ongoing activity'},
        ]},
        {'name':'Patterns not considered abnormal', 'children':[
            {'name':'Physiologic patterns', 'children':[
                {'name':'Rhythmic activity'},
                {'name':'Slow alpha variant rhythms'},
                {'name':'Fast alpha variant rhythms'},
                {'name':'Frontocentral theta activity'},
                {'name':'Lambda waves'},
                {'name':'Posterior slow waves in youth'},
                {'name':'Diffuse low frequency activity induced by hyperventilation'},
                {'name':'Photic drive response'},
                {'name':'Photomyogenic response(orbitofrontal photomyoclonus)'},
                {'name':'Arousal pattern'},
                {'name':'Frontal arousal rhythm'},
            ]},
            {'name':'Patterns of uncertain significance', 'children':[
                {'name':'Sharp transient Wicket spikes'},
                {'name':'Small sharp spikes (Benign Epileptiform Transients of Sleep)'},
                {'name':'Rhythmic temporal theta burst of drowsiness'},
                {'name':'Ciganek rhythm (midline central theta)'},
                {'name':'6 Hz spike and slow wave'},
                {'name':'14 and 6 Hz positive bursts'},
                {'name':'Rudimentary spike wave complex'},
                {'name':'Slow fused transient'},
                {'name':'Needle-like occipital spikes of the blind'},
                {'name':'Subclinical Rhythmic EEG Dischargesin Adults (SREDA)'},
                {'name':'Temporal slowing in elderly subjects Breach rhythm'},
            ]},
        ]},
    ]}
    
    

In [292]:
"""def preorder_label_parent(tree_dict, labels=None, parents=None):
    if labels is None:
        labels=list()
    if parents is None:
        parents=[''] 
    labels.append(tree_dict['name'])
    last_node=tree_dict['name']
    if 'children' in tree_dict:
        for child in tree_dict.get('children'):
            parents.append(last_node)
            preorder_label_parent(child, labels, parents)
        
    return labels, parents"""

In [293]:
"""pre_labels, pre_parents = preorder_label_parent(eeg_terminiology)
list(zip(pre_labels, pre_parents))"""

[('Root_EEG', ''),
 ('Location of graphelements', 'Root_EEG'),
 ('laterality', 'Location of graphelements'),
 ('left', 'laterality'),
 ('right', 'laterality'),
 ('midline', 'laterality'),
 ('bilateral', 'laterality'),
 ('diffuse', 'laterality'),
 ('region', 'Location of graphelements'),
 ('frontal', 'region'),
 ('perisylvian-superior surface', 'frontal'),
 ('lateral', 'frontal'),
 ('mesial', 'frontal'),
 ('polar', 'frontal'),
 ('orbitofrontal', 'frontal'),
 ('central', 'region'),
 ('lateral convexity', 'central'),
 ('mesial', 'central'),
 ('central sulcus -anterior surface', 'central'),
 ('central sulcus -posterior surface', 'central'),
 ('opercular', 'central'),
 ('temporal', 'region'),
 ('polar', 'temporal'),
 ('basal', 'temporal'),
 ('lateral-anterior', 'temporal'),
 ('lateral-posterior', 'temporal'),
 ('perisylvian-inferior surface', 'temporal'),
 ('parietal', 'region'),
 ('lateral-convexity', 'parietal'),
 ('mesial', 'parietal'),
 ('opercular', 'parietal'),
 ('occipital', 'region'

In [27]:
from collections import deque

def level_parent_label(json_data,  labels=None, parents=None):
    """The function level_parent_label() extracts the label, parent lists in the level order:"""
    
    if labels is None:
        labels = list()
    if parents is None:
        parents = ['']
    to_traverse = deque([json_data])
    
    while len(to_traverse) > 0:
        subtree = to_traverse.popleft()
        #print('\n', subtree)  # uncomment this line to see the contents of the dict subtree
        if 'name' in subtree:
            labels.append(subtree['name']) # subtree['name'] is the name of the subtree root
            parent = subtree['name']
            if 'children' in subtree:
                for child in subtree['children']:
                    parents.append(parent)
                to_traverse.extend(subtree['children'])
    return  labels, parents


In [28]:
level_labels, level_parents = level_parent_label(eeg_terminiology)
list(zip(level_labels, level_parents))

[('Root_EEG', ''),
 ('Location of graphelements', 'Root_EEG'),
 ('Interical findings', 'Root_EEG'),
 ('Ictal EEG findings', 'Root_EEG'),
 ('Patterns not considered abnormal', 'Root_EEG'),
 ('laterality', 'Location of graphelements'),
 ('region', 'Location of graphelements'),
 ('Epileptiform interictal activity', 'Interical findings'),
 ('Abnormal interictal rhythmic activity', 'Interical findings'),
 ('Special patterns', 'Interical findings'),
 ('No observable change', 'Ictal EEG findings'),
 ('Obscured by artifacts', 'Ictal EEG findings'),
 ('Polyspikes', 'Ictal EEG findings'),
 ('Fast spike activity or repetitive spikes', 'Ictal EEG findings'),
 ('Low voltage fast activity', 'Ictal EEG findings'),
 ('Polysharp-waves', 'Ictal EEG findings'),
 ('Spike-and-slow-waves', 'Ictal EEG findings'),
 ('Polyspike-and-slow-waves', 'Ictal EEG findings'),
 ('Sharp-and-slow-waves', 'Ictal EEG findings'),
 ('Rhythmic activity', 'Ictal EEG findings'),
 ('Slow wave of large amplitude', 'Ictal EEG findi

In [29]:
def get_ids(labels, parents):
    if len(labels) !=  len(parents):
        raise ValueError('The list of labels should have the same length like the list of parents')
    N = len(labels)
    ids =[str(id)   for id in range(N)]
    dlabels = {label: idx  for label, idx in zip(labels, ids)} # associate to each label the corresponding id
    parentids =[''] + [dlabels[label] for label in parents[1:]] 
    return ids, parentids

In [30]:
labels, parents = level_parent_label(eeg_terminiology)

In [31]:
ids, parentids = get_ids(labels, parents)

In [32]:
len(labels), len(ids), len(parentids)

(110, 110, 110)

In [33]:
flare_trace = go.Sunburst(ids=ids,
                          labels=labels,
                          parents=parentids)

In [35]:
layout = go.Layout(title=dict(text='Example of EEG Findings Terminology', x=0.5),
                   font= dict(size=12),
                   showlegend=False,
                   autosize=False,
                   width=950,
                   height=950,
                   xaxis=dict(visible=False),
                   yaxis=dict(visible=False),          
                   hovermode='closest'
                  )

In [320]:
# Save file
fig1.write_html("eeg_terminology_plot.html")